In [22]:
from pathlib import Path
import urllib.request
import tarfile
import pandas as pd
import numpy as np
from typing import Tuple
from scipy.stats import randint, uniform, loguniform, expon

# Sklearn import
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils.validation import check_is_fitted, check_array
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import clone

## Exercice 1
Use a SVR Model on 5000 elem to train it and see how it goes

In [23]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as tarball:
            tarball.extractall(path="datasets")
    return pd.read_csv("datasets/housing/housing.csv")

housing_downloaded = load_housing_data()

In [24]:
housing = housing_downloaded.copy()
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [25]:
def get_strat_train_and_test_split(data: pd.DataFrame, stratified_column_name: str, train_size: float=0.8, delete_strat_col: bool = False) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_set, test_set = train_test_split(data, train_size=train_size, stratify=housing[stratified_column_name], random_state=42)
    if delete_strat_col:
        train_set.drop(stratified_column_name, axis=1, inplace=True)
        test_set.drop(stratified_column_name, axis=1, inplace=True)
    return train_set, test_set

In [26]:
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0., 1.5, 3., 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])

In [27]:
housing_train_set, housing_test_set = get_strat_train_and_test_split(housing, "income_cat", train_size=0.2, delete_strat_col=True) # Try it on a lower train set
housing_train_data = housing_train_set.drop("median_house_value", axis=1)
housing_train_labels= housing_train_set["median_house_value"].copy()

Import Data pipelines uses through the chapter

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state, n_init=10)

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [29]:
def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_name_in):
    return["ratio"]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler()
    )

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler()
)

cluster_simil = ClusterSimilarity(random_state=42)
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
    ("geo", cluster_simil, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object))
], remainder=default_num_pipeline)

In [30]:
svr_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("svr", SVR()),
])
param_grid = [
    {'svr__kernel': ['linear'], 'svr__C': [10., 30., 100., 300., 1000.,
                                           3000., 10000., 30000.0]},
    {'svr__kernel': ['rbf'], 'svr__C': [1.0, 3.0, 10., 30., 100., 300.,
                                        1000.0],
     'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]
grid_search = GridSearchCV(svr_pipeline, param_grid, cv=3,
                           scoring='neg_root_mean_squared_error', verbose=3)
grid_search.fit(housing_train_data, housing_train_labels)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV 1/3] END svr__C=10.0, svr__kernel=linear;, score=-107387.381 total time=   0.3s
[CV 2/3] END svr__C=10.0, svr__kernel=linear;, score=-111164.581 total time=   0.3s
[CV 3/3] END svr__C=10.0, svr__kernel=linear;, score=-105530.404 total time=   0.3s
[CV 1/3] END svr__C=30.0, svr__kernel=linear;, score=-95211.466 total time=   0.3s
[CV 2/3] END svr__C=30.0, svr__kernel=linear;, score=-102210.342 total time=   0.2s
[CV 3/3] END svr__C=30.0, svr__kernel=linear;, score=-92963.638 total time=   0.3s
[CV 1/3] END svr__C=100.0, svr__kernel=linear;, score=-83764.156 total time=   0.3s
[CV 2/3] END svr__C=100.0, svr__kernel=linear;, score=-103078.978 total time=   0.3s
[CV 3/3] END svr__C=100.0, svr__kernel=linear;, score=-81284.939 total time=   0.3s
[CV 1/3] END svr__C=300.0, svr__kernel=linear;, score=-77678.135 total time=   0.3s
[CV 2/3] END svr__C=300.0, svr__kernel=linear;, score=-103368.280 total time=   0.3s
[CV 3/3] END s

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x136e6fd50>)])),
                                       ('svr', SVR())]),
             param_grid=[{'svr__C': [10.0, 30.0, 100.0, 300.0, 1000.0, 3000.0,
                                     10000.0, 30000.0],
                          'svr__kernel': ['linear']},
                         {'svr__C': [1.0, 3.0, 10.0, 30.0, 100.0, 300.0,
                                     1000.0],
                          'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0],
                          'svr__kernel': ['rbf']}],
             scoring='neg_root_mean_squared_error', verbose=3)

In [31]:
grid_search.best_params_

{'svr__C': 10000.0, 'svr__kernel': 'linear'}

In [32]:
grid_search.best_score_

-74831.88140755171

## Exercice 2
Use the RandomSearchCV instead of GridSearchCV

In [33]:
param_distribs = {
    'svr__kernel': ['linear', 'rbf'], 'svr__C': randint(low=0, high=30000),
    'svr__gamma': uniform.rvs(loc=0.1, scale=2.9, size=100),
}
# The one from the correction
param_distribs_bis = {
    'svr__kernel': ['linear', 'rbf'],
    'svr__C': loguniform(20, 200_000),
    'svr__gamma': expon(scale=1.0),
}
rand_seach = RandomizedSearchCV(svr_pipeline, param_distribs, n_iter=50, scoring='neg_root_mean_squared_error', cv=3, verbose=3)
rand_seach.fit(housing_train_data, housing_train_labels)
rand_seach_bis = RandomizedSearchCV(svr_pipeline, param_distribs_bis, n_iter=50, scoring='neg_root_mean_squared_error', cv=3, verbose=3)
rand_seach_bis.fit(housing_train_data, housing_train_labels)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV 1/3] END svr__C=11079, svr__gamma=2.3661350122964935, svr__kernel=rbf;, score=-107838.681 total time=   0.4s
[CV 2/3] END svr__C=11079, svr__gamma=2.3661350122964935, svr__kernel=rbf;, score=-112408.451 total time=   0.4s
[CV 3/3] END svr__C=11079, svr__gamma=2.3661350122964935, svr__kernel=rbf;, score=-109182.885 total time=   0.4s
[CV 1/3] END svr__C=21819, svr__gamma=1.2179000263985422, svr__kernel=rbf;, score=-87532.411 total time=   0.4s
[CV 2/3] END svr__C=21819, svr__gamma=1.2179000263985422, svr__kernel=rbf;, score=-90541.938 total time=   0.4s
[CV 3/3] END svr__C=21819, svr__gamma=1.2179000263985422, svr__kernel=rbf;, score=-89611.874 total time=   0.4s
[CV 1/3] END svr__C=8700, svr__gamma=2.4157254500632095, svr__kernel=rbf;, score=-109799.626 total time=   0.4s
[CV 2/3] END svr__C=8700, svr__gamma=2.4157254500632095, svr__kernel=rbf;, score=-114146.125 total time=   0.3s
[CV 3/3] END svr__C=8700, svr__gamma=2.

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('bedrooms',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('functiontransformer',
                                                                                                FunctionTransformer(feature_names_...
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x136e6fd50>)])),
                                             ('svr', SVR())]),
                   n_iter=50,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x2bd9f85d0>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x135dcebd0>,
                                        'svr__kernel': ['linear', 'rbf']},
                   scoring='neg_root_mean_squared_error', verbose=3)

In [34]:
rand_seach.best_score_, rand_seach.best_params_, rand_seach_bis.best_score_, rand_seach_bis.best_params_

(-65602.62414836553,
 {'svr__C': 11094, 'svr__gamma': 0.13283074360251, 'svr__kernel': 'rbf'},
 -58978.23077266538,
 {'svr__C': 167861.22130054102,
  'svr__gamma': 0.0959304252723388,
  'svr__kernel': 'rbf'})

## Exercices 3
Adding a SelectFromModel transformer in the Pipelines

In [35]:
svr_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("selector", SelectFromModel(RandomForestRegressor(), threshold=0.005)),
    ("svr", SVR(C=103500.3865094702, gamma=0.4390901526150056, kernel='rbf')),
])

selector_rmses = -cross_val_score(svr_pipeline, housing_train_data, housing_train_labels, cv=10, scoring='neg_root_mean_squared_error', verbose=2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   3.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.9s remaining:    0.0s


[CV] END .................................................... total time=   3.8s
[CV] END .................................................... total time=   3.6s
[CV] END .................................................... total time=   3.6s
[CV] END .................................................... total time=   3.8s
[CV] END .................................................... total time=   3.6s
[CV] END .................................................... total time=   3.8s
[CV] END .................................................... total time=   3.6s
[CV] END .................................................... total time=   3.7s
[CV] END .................................................... total time=   3.5s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   37.1s finished


In [36]:
pd.Series(selector_rmses), np.mean(selector_rmses)

(0    57766.577497
 1    56564.802079
 2    69490.793590
 3    50626.430950
 4    60739.919844
 5    61960.833817
 6    62568.208802
 7    59081.376349
 8    61680.160911
 9    54887.251767
 dtype: float64,
 59536.63556075973)

## Exercice 4
Create a custom transformer that that train a k-nearest neighbors regressor in tis `fit()` and outputs the model's prediction in its `transform()` methods

In [37]:
class KNNCustomTransfomer(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5, weights="uniform"):
        self.n_neighbors = n_neighbors
        self.weights = weights

    def fit(self, X, y=None):
        self.knn_ = KNeighborsRegressor(n_neighbors=self.n_neighbors, weights=self.weights)
        self.knn_.fit(X, y)
        self.n_features_in_ = self.knn_.n_features_in_
        if hasattr(self.knn_, "feature_names_in_"):
            self.feature_names_in_ = self.knn_.feature_names_in_
        return self

    def transform(self, X):
        check_is_fitted(self)
        predictions = self.knn_.predict(X)
        if predictions.ndim == 1:
            return predictions.reshape((-1, 1))
        return predictions

    def get_feature_names_out(self, names=None):
        return [f"{self.knn_.__class__.__name__.lower().replace('_', '')}_prediction"]

In [38]:
check_estimator(KNNCustomTransfomer(n_neighbors=5))

In [39]:
knn_transformer = KNNCustomTransfomer()
geo_features = housing_train_data[["latitude", "longitude"]]
knn_transformer.fit_transform(geo_features, housing_train_labels)

array([[213400.],
       [ 65660.],
       [110720.],
       ...,
       [266720.],
       [ 73560.],
       [ 71760.]])

In [40]:
knn_transformer.get_feature_names_out()

['kneighborsregressor_prediction']

In [41]:
transformers = [(name, clone(transformer), columns) for name, transformer, columns in preprocessing.transformers]
geo_transformers_idx = [name for name, _, __ in transformers].index("geo")
transformers[geo_transformers_idx] = ("geo", KNNCustomTransfomer(), ["latitude", "longitude"])

new_geo_preprocessing = ColumnTransformer(transformers)

In [42]:
svr_pipeline = Pipeline([
    ("preprocessing", new_geo_preprocessing),
    ("svr", SVR(C=103500.3865094702, gamma=0.4390901526150056, kernel='rbf')),
])

svr_rmses = -cross_val_score(svr_pipeline, housing_train_data, housing_train_labels, cv=10, scoring='neg_root_mean_squared_error', verbose=2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.4s finished


In [43]:
pd.Series(svr_rmses), np.mean(svr_rmses)

(0     95936.827665
 1    103761.464874
 2    108469.247383
 3    103416.245991
 4    102907.585404
 5    108042.136016
 6    104425.571108
 7    103217.486975
 8    105463.730974
 9    100171.243910
 dtype: float64,
 103581.15402988189)

In [44]:
estimator = svr_pipeline.fit(housing_train_data, housing_train_labels)

In [45]:
svr_pipeline.named_steps

{'preprocessing': ColumnTransformer(transformers=[('bedrooms',
                                  Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer(strategy='median')),
                                                  ('functiontransformer',
                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x2bdab1760>,
                                                                       func=<function column_ratio at 0x2bdab1440>)),
                                                  ('standardscaler',
                                                   StandardScaler())]),
                                  ['total_bedrooms', 'total_rooms']),
                                 ('rooms_per_house',
                                  Pipeline(s...
                                  ['total_bedrooms', 'total_rooms', 'population',
                                   'households', 'median_income']),
  

## Exercice 5
Use RandomSearchCV on the new pipeline

In [46]:
param_distribs = {
    "preprocessing__geo__n_neighbors": range(1, 30),
    "preprocessing__geo__weights": ["distance", "uniform"],
    "svr__C": loguniform(20, 200_000),
    "svr__gamma": expon(scale=1.0),
}
rand_seach = RandomizedSearchCV(svr_pipeline, param_distribs, n_iter=200, cv=3, scoring='neg_root_mean_squared_error', verbose=2)
rand_seach.fit(housing_train_data, housing_train_labels)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
[CV] END preprocessing__geo__n_neighbors=7, preprocessing__geo__weights=uniform, svr__C=6767.9938554682985, svr__gamma=1.3531909410585576; total time=   0.2s
[CV] END preprocessing__geo__n_neighbors=7, preprocessing__geo__weights=uniform, svr__C=6767.9938554682985, svr__gamma=1.3531909410585576; total time=   0.2s
[CV] END preprocessing__geo__n_neighbors=7, preprocessing__geo__weights=uniform, svr__C=6767.9938554682985, svr__gamma=1.3531909410585576; total time=   0.2s
[CV] END preprocessing__geo__n_neighbors=17, preprocessing__geo__weights=distance, svr__C=100176.07016702631, svr__gamma=2.6100598407925943; total time=   0.3s
[CV] END preprocessing__geo__n_neighbors=17, preprocessing__geo__weights=distance, svr__C=100176.07016702631, svr__gamma=2.6100598407925943; total time=   0.4s
[CV] END preprocessing__geo__n_neighbors=17, preprocessing__geo__weights=distance, svr__C=100176.07016702631, svr__gamma=2.6100598407925943; to

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('bedrooms',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('functiontransformer',
                                                                                                FunctionTransformer(feature_names_out=<function ratio_name at 0x2bdab1760>,
                                                                                                                    func=<function column_ratio at 0x2bdab1440>)),
                                                                                               ('standardscaler',
                                                                                                StandardScaler()...
                   n_iter=200,
                   param_distributions={'preprocessing__geo__n_neighbors': range(1, 30),
                                        'preprocessing__geo__weights': ['distance',
                                                                        'uniform'],
                                        'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x2b972f190>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x2bdb2cf10>},
                   scoring='neg_root_mean_squared_error', verbose=2)

In [47]:
rand_seach.best_score_, rand_seach.best_params_

(-72137.2662361325,
 {'preprocessing__geo__n_neighbors': 1,
  'preprocessing__geo__weights': 'distance',
  'svr__C': 156935.95947236067,
  'svr__gamma': 0.0023124367527386307})

## Exercice 6
Re-implement the StandardScaler

In [48]:
class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean: bool = True) -> None:
        self.with_mean = with_mean

    def fit(self, X: pd.DataFrame | np.ndarray, y=None):
        if hasattr(X, "columns"):
            self.feature_names_in_ = X.columns
        X = check_array(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        self.n_features_in_ = X.shape[1]

        return self

    def transform(self, X: pd.DataFrame | np.ndarray, y=None) -> np.ndarray:
        check_is_fitted(self)
        X = check_array(X)
        if self.n_features_in_ != X.shape[1]:
            raise ValueError("Error on numbers of features")
        if self.with_mean:
            X = X - self.mean_
        return X / self.std_

    def inverse_transform(self, X: pd.DataFrame | np.ndarray) -> np.ndarray:
        check_is_fitted(self)
        X = check_array(X)
        if self.n_features_in_ != X.shape[1]:
            raise ValueError("Error on numbers of features")
        if not self.with_mean:
            return X * self.std_
        return X * self.std_ + self.mean_

    def get_feature_names_out(self, input_features: list | None = None) -> list:
        if input_features is None:
            if hasattr(self, "feature_names_in_"):
                return self.feature_names_in_
            return [f"x{i}" for i in range(self.n_features_in_)]
        if len(input_features) != self.n_features_in_:
            raise ValueError("Number of elem in the input_features doesn't match with the len of inputed features")
        if (features_name := hasattr(self, "feature_names_in_")) and not np.all(self.feature_names_in_ == input_features):
            raise ValueError("input_features doesn't match with feature_names_in_")
        return input_features

## Check from the correction

In [49]:
check_estimator(StandardScalerClone())

In [50]:
np.random.seed(42)
X = np.random.rand(1000, 3)

scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(X)

assert np.allclose(X_scaled, (X - X.mean(axis=0)) / X.std(axis=0))

In [51]:
scaler = StandardScalerClone(with_mean=False)
X_scaled_uncentered = scaler.fit_transform(X)

assert np.allclose(X_scaled_uncentered, X / X.std(axis=0))

In [52]:
scaler = StandardScalerClone()
X_back = scaler.inverse_transform(scaler.fit_transform(X))

assert np.allclose(X, X_back)

In [53]:
assert np.all(scaler.get_feature_names_out() == ["x0", "x1", "x2"])
assert np.all(scaler.get_feature_names_out(["a", "b", "c"]) == ["a", "b", "c"])

In [54]:
df = pd.DataFrame({"a": np.random.rand(100), "b": np.random.rand(100)})
scaler = StandardScalerClone()
X_scaled = scaler.fit_transform(df)

assert np.all(scaler.feature_names_in_ == ["a", "b"])
assert np.all(scaler.get_feature_names_out() == ["a", "b"])